<a href="https://colab.research.google.com/github/FrancescoCortopassi/copilot-codespaces-vscode/blob/main/MT5_LEZIONE_57_to_65.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEZIONE_57: STRATEGY DEPLOIMENT INTRO
Prima di implementare la nostra strategia in real dobbiamo prendere in considerazione alcuni punti importanti:


1. POSITION SIZE: Questo parametro deciderà qual'è la size della nostra posizione e quanto possiamo aspettarci in termini di P&L (profitti e perdite). Consigliabile ottimizzare questo parametro testandolo sulla startegia e sul portafoglio per considerare l'impatto sul nostro ecosistema di trading.
2. ASSETS: Quale/quali assets vogliamo includere nel nostro portafoglio per questa strategia? Tutti gli assets hanno gli stessi parametri?(ex: timeframe). In più ogni asset potrebbe avere più o meno parametri (dobbiamo quindi comprendere anche come fornire differenti parametri agli asset ed il corso non approfondisce questo punto)
3. NOTIFICATIONS: È imortante rimanere aggiornati sullo svolgimento del nostro trading per monitorare attivamente alcuni parametri (punti di ingresso e di uscita, money mangment,account balance ed altro)
4. CONTINGENCY: Quali misure abbiamo preso per evitare che la strategia produca perdite minimali e non ci siano problematiche di disconnessione o altro genere?



LEZIONE_58: POSITION SIZE


1. La scelta della size è un fattore determinante per il successo di una strategia
2. Ci sono molteplici approcci alla gestione delle size: Fixed, Risk PnL basaed, sofisticated models etc
3. Quello che noi andremo ad implementare nelal nostra strategia sarà basata su Risk PnL e punterà ad avere lo stesso (o quasi) ratio di vincite e perdite per movimento di pip (DA APPROFONDIRE IL CONCETTO PERCHé NON COMPRESO PERFETTAMENTE)



LEZIONE_59: POSITION SIZE RISK MANAGMENT
Andiamo a vedere come sviluppare il codice per ottimizzare la size della nostra posizione in base al nostro programma di gestione del rischio. Nel codice del corso si utilizza come valuta di benchmark il dollaro statunitense (USD). Lo stesso concetto può essere applocato a qualsiasi moneta.

In [ ]:
#LEZIONE_59: POSITION SIZE RISK MANAGMENT

# establish MetaTrader 5 connection to a specified trading account
if mt5.initialize():
    print("connection established")

time.sleep(2)

#get current price of a symbol
def get_current_price(symbol):
    tick_info = mt5.symbol_info_tick(symbol)
    return (tick_info.bid + tick_info.ask)/2

def usd_conversion_factor(currency):
    if currency == "USD":
        return 1

    currency_data = mt5.symbols_get("*{}*".format(currency))
    for symbol in currency_data:
        if symbol.name[:3] == "USD" or symbol.name[3:] == "USD":
            base_currency = symbol.name[:3]
            counter_currency = symbol.name[3:]
            break
    currency_data = mt5.symbol_info(base_currency+counter_currency)
    current_price = get_current_price(base_currency+counter_currency)
    if base_currency == "USD":
        return 1/current_price
    else:
        return current_price


LEZIONE_60: CORRELATION ANALYSIS
Questo passsaggio è una cosa a cui io credo molto per mia natura, ovvero l'nalisi della correlazione tra due coppie di valute. la correlazione misura il coefficiente di "simbiosi" rappresentato da due valute, in poche parole quanto queste valute si muovono nello stesso modo durante il tempo.

C'è un modo statistico di identificare la correlazione, il PEARSON CORRELATION COEFFICIENT (che andremo a vedere nel codice). Se la correlazione è superiore a 1 significa che c'è una grande correlzione, vicino allo 0 significa che non c'è correlazione e vicino al -1 significa che c'è correlazione negativa.

STRATEGIA DI PORTAFOGLIO: Avere un paniere di assets con una bassa correlazione permette al nostro portafoglio di mantenersi più stabile e di non essere sbilanciato in periodi di alta volatilità

In [ ]:

# -*- coding: utf-8 -*-
"""
MetaTrader5 - Correlation analysis

@author: Mayank
"""

import MetaTrader5 as mt5
import os
import datetime as dt
import pandas as pd
import pytz

os.chdir(r"C:\Users\Mayank\OneDrive\Udemy\MT5 Algorithmic Trading") #path where login credentials and server details
key = open("key.txt","r").read().split()
path = r"C:\Program Files\MetaTrader 5\terminal64.exe"


# establish MetaTrader 5 connection to a specified trading account
if mt5.initialize(path=path,login=int(key[0]), password=key[1], server=key[2]):
    print("connection established")

#extract historical data
def get_hist_data(symbol, timeframe, time_till=None, num_candles=200):
    """
    Parameters
    ----------
    symbol : TYPE str - e.g "USDCAD"
    timeframe : TYPE str - e.g. "TIMEFRAME_M15"
    time_till : TYPE str -e.g. "YYYY-MM-DD HH:MM:SS"
    num_candles : TYPE int

    Returns
    -------
    historical data dataframe

    """
    #pytz.all_timezones
    current_tz = pytz.timezone("Asia/Kolkata") #change this based on your location
    eet_tz = pytz.timezone("Europe/Kyiv")
    required_tz = pytz.timezone("Etc/UTC")

    if time_till == None:
        time_till = current_tz.localize(dt.datetime.now()).replace(tzinfo=required_tz)
    else:
        time_till = eet_tz.localize(dt.datetime.strptime(time_till, "%Y-%m-%d %H:%M:%S")).replace(tzinfo=required_tz)

    hist_data = mt5.copy_rates_from(symbol, getattr(mt5, timeframe), time_till, num_candles)
    hist_data_df = pd.DataFrame(hist_data)
    hist_data_df.time = pd.to_datetime(hist_data_df.time, unit="s")
    hist_data_df.set_index("time", inplace=True)
    return hist_data_df

#hist_data = get_hist_data("USDCAD", "TIMEFRAME_M30") #get data till current time

# symbols = mt5.symbols_get()
# symbols = [i.name for i in symbols if "Forex" in i.path]

symbols = ["USDCAD","USDJPY","GBPUSD","EURUSD","AUDUSD","USDCHF","AUDNZD","NZDJPY","AUDCHF","GBPMXN","USDSEK","NZDUSD"]

hist_data_cp_df = pd.DataFrame()
for symbol in symbols:
    temp = get_hist_data(symbol, "TIMEFRAME_D1", num_candles=1000)
    hist_data_cp_df[symbol] = temp.close



hist_data_cp_df_ret = hist_data_cp_df.pct_change()
hist_data_cp_df_ret.dropna(inplace=True)
corr_matrix = hist_data_cp_df_ret.corr()
cm_below_threshold = corr_matrix[abs(corr_matrix) < 0.3]


#Il comando CORR ci fornisce come argomento di default la correlazione PEARSON, ma possiamo
#passare come argomento (parametro) qualsiasi altro metodo di correlazione presente nella libreria
#pandas. Quello che fa questo metodo è calcolare la correlazione basandosi sui ritorni (returns),
#che poi è l'unico modo fuznionale di poter correlare le valute in un dataFrame di pandas
#Per comodità memorizzioamo il codice di correlazione nella variabile CORR_MATRIX

#NOTA:Le correlazioni sono molto utilizzate dai trader sul forex per ottimizzare il portafoglio
#(cercando la non correlazione). Nelle strategie di PAIR TRADING(come l'arbitraggio su PAIR) andiamo acercare
#coppie altamente correlate per aprire operazioni durante la fase di non correlazione, che solitamente
#precede un ritorno alla media (MEAN REVERSION)

LEZIONE_61: SEND EMAILS NOTIFICATIONSpython fornisce ai propri programamtori molti modi di creare notifiche e ci sono diverse librerie davvero funzionali per creare questo tipo di allert (via email, con messaggio telefonico, via telegram o whatsapp)

In questo caso andremo at utilizzare le notifiche via email perché risultano più affidabili e possono contenere più testo ed informazioni in maniera più fruibile rispetto ad altri tipi di notifica.
Si può utilizzare qualsiasi provider (noi useremo Gmail) ma alcuni provider sono più difficli da implementare rispetto ad altri perché sono proprio i provider a rendere le operazioni di autenticazioni piuttosto complicate

In [ ]:
Sending email notifications using smtplib

@author: Mayank


import MetaTrader5 as mt5
import os
import time
import smtplib

def send_email(message, email_id="abc@gmail.com", app_password="blah blah", to_email_id="xyz@gmail.com"):
    try:
        s = smtplib.SMTP('smtp.gmail.com', 587)
        s.starttls()
        s.login(email_id,app_password)
        email_subject="MT5 TradingBot_1 Notification"
        email_body=message
        s.sendmail(email_id,to_email_id, 'Subject: {}\n\n{}'.format(email_subject, email_body))
        s.quit()
    except Exception as e:
        print(e)

os.chdir(r"C:\Users\Mayank\OneDrive\Udemy\MT5 Algorithmic Trading") #path where login credentials and server details
key = open("key.txt","r").read().split()
path = r"C:\Program Files\MetaTrader 5\terminal64.exe"


# establish MetaTrader 5 connection to a specified trading account
if mt5.initialize(path=path,login=int(key[0]), password=key[1], server=key[2]):
    print("connection established")
    send_email("connection established")

time.sleep(2)

LEZIONE_62_TO_65: STRATEGY DEPLOYMENT
In queste lezioni andiamo a vedere come implementare la strategia che abbiamo backtestato nel capitolo precedente del corso.

NOTA: Quello che ho compreso è che si può sviluppare codice per una strategia inserendolo in fase di sviluppo direttamente all'interno di un contesto di backtesting. Questo implica un'ulteriore modifica nel momento in cui vogliamo implementare la strategia in live: dovremo quindi riscrivere il codice esulandolo dalla parte riguardante iol backtest (queste sono impressioni che DEVO APPROFONDIRE IN FASE DI RIPASSO).

SVOLGIMENTO: Per questa parte del corso ho optato per l'utilizzo dei files .PY che sono stati messi a disposizione dal corso per non dover riscrivere tutto il codice. Quindi di seguito caricherò tutto il codice fornito in questo capitolo per poterlo studiare senza impiegare tempo nella trascrizione.

In [3]:
!python3 /content/macd_supp_res_strat.py

Traceback (most recent call last):
  File "/content/macd_supp_res_strat.py", line 8, in <module>
    import MetaTrader5 as mt5
ModuleNotFoundError: No module named 'MetaTrader5'


ERROR: Non sono riuscito a caricare i file.py che mi servivano ma ho comunque trascritto il codice base per caricare i files.py qui su Colab (utilizzando il comando "!python3"). In seguito sarà mia premura comprendere come lavorare con MT5 su questa piattaforma che io prediligo per l'assistenza al codice